In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import sqrt
from collections import OrderedDict

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [2]:
transactions = spark.read.option("header", "true").option("inferschema", "true").option("mode", "DROPMALFORMED").csv("transaction_customer_join.csv")
transactions.rdd.getNumPartitions()

7

In [3]:
values = transactions.select("latitude", "longitude", "currencyAmount", "date", "age", "gender", "totalIncome").collect()

In [6]:
%%time

weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

areas = []

lat_half_interval = 1.5/111
lon_half_interval = 1.5/111

def floorHour(hour):
    # need to make new time ranges?
    if 6.0 <= hour <= 11.0:
        return 'morning'
    
    elif 11.0 < hour <= 17.0:
        return 'noon'

    elif 17.0 < hour < 22.0:
        return 'evening'
    
    else:
        return 'other'
    

for (latitude, longitude, currencyAmount, date, age, gender, income) in values:
    
    if latitude is not None and longitude is not None:
    
        i = 0
        
        areaExists = False

        weekday = weekdays[date.weekday()]
        
        for area in areas:
            if area["min_lat"] < latitude < area["max_lat"] and area["min_lon"] < longitude < area["max_lon"]:
                areaExists = True

                area["totalIncome"] += income
                area[weekday]["numTransactions"] += 1
                area[weekday]["totalSpent"] += currencyAmount
                area[weekday]["totalAge"] += age
                area[weekday][gender] += 1
                area[weekday][floorHour(date.hour)] += 1
                
                
        if not areaExists:
            newArea = {
                "min_lat": latitude-lat_half_interval, "max_lat": latitude+lat_half_interval,
                "min_lon": longitude-lon_half_interval, "max_lon": longitude+lon_half_interval,
                "totalIncome": income
            }
            for day in weekdays:
                newArea[day] = {
                   "morning":0, "noon":0, "evening":0, "other":0,
                   "Male":0, "Female":0, "Other":0,
                   "totalAge":0,
                   "numTransactions":0,
                   "totalSpent":0
                }

            newArea[weekday]["numTransactions"] += 1
            newArea[weekday]["totalSpent"] += currencyAmount
            newArea[weekday]["totalAge"] += age
            newArea[weekday][gender] += 1
            newArea[weekday][floorHour(date.hour)] += 1
                
            areas.append(newArea)
                
len(areas)

Wall time: 2.08 s


In [7]:
%%time

with open("Aggregated.json", "w") as file:
    file.write(str(areas))

Wall time: 25.9 ms
